## `Практика PySpark` создание пайплайна

**Задача**. Сделать пайплайн обработки файла `cars.csv`. 

Необходимо посчитать по каждому производителю (поле `manufacturer_name`):
- `кол-во` объявлений
- `средний год выпуска` автомобилей
- `минимальную цену`
- `максимальную цену`

Выгрузить результат в `output.csv`.

In [1]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as t
from fields import *


def extractor(spark: SparkSession, path: str) -> DataFrame:
    """Вернет дф, прочитанный 
        по переданному пути. 
        Если путь не передан =
        = прочтет файл дефолта """
    if not path:
        path = 'data/cars.csv'
    return spark.read.option('header', 'true').csv(path)


def outputer(df: DataFrame) -> DataFrame:
    """Обработает полученный дф
        Применям агрегирующие 
        функции, согласно задачам """
    # output =
    return df.groupBy(manufacturer_name).agg(
        # считаем объявления
        F.count(manufacturer_name).alias("Кол-во объявлений"),
        # средний год выпуска машины. Не забыть окуглить 
        # до целых и привести к типу int через .t
        F.round(F.avg(year_produced)).cast(
            t.IntegerType()).alias("Средний год выпуска авто"),
        F.min(price_usd).alias("Минимальная цена"),
        F.max(price_usd).alias("Максимальная цена"),
    ).orderBy(F.col(manufacturer_name).desc())
    # return output


def saver (df: DataFrame, name_fold ) -> None:
    """ Сохранит полученные данные после агрегации """
    df.coalesce(2).write.mode("overwrite").format("json").save(name_fold)
    

def runer():
    """Стартует сессию """
    spark = SparkSession.builder.appName("Практика").getOrCreate()
    if saver(outputer(extractor(spark,'')),"output.json.test"):
        spark.stop()


if __name__ == "__main__":
    runer()
   

21/12/07 11:50:29 WARN Utils: Your hostname, vostrolpt resolves to a loopback address: 127.0.1.1; using 172.21.113.208 instead (on interface eth0)
21/12/07 11:50:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/07 11:50:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
